In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

# Step i. Load a small dataset (CIFAR-10)
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Step ii. Split the dataset
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

# Step iii. Visualize some samples
def imshow(img, labels, class_names):
    img = img / 2 + 0.5  # Unnormalize
    npimg = img.numpy()
    plt.figure(figsize=(10, 5))
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.title("Labels: " + ", ".join([class_names[labels[j]] for j in range(len(labels))]))
    plt.show()

class_names = train_dataset.classes
dataiter = iter(train_loader)
images, labels = next(dataiter)
imshow(torchvision.utils.make_grid(images[:8]), labels[:8], class_names)

# Step iv. Define CNN architecture with feature map functionality
class SimpleCNN(nn.Module):
    def _init_(self):
        super(SimpleCNN, self)._init_()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)  # Convolutional layer
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.fc1 = nn.Linear(32 * 8 * 8, 128)  # Fully connected layer
        self.fc2 = nn.Linear(128, 10)  # Output layer
        
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(2, 2)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        feature_map = x  # Capture feature map after conv1
        x = self.pool(feature_map)
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(-1, 32 * 8 * 8)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x, feature_map  # Return the feature map for visualization

# Initialize model
model = SimpleCNN()

# Step v. Loss function
criterion = nn.CrossEntropyLoss()

# Step vi. Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Step vii. Train the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
num_epochs = 5

train_losses = []
train_accuracies = []
test_losses = []
test_accuracies = []

for epoch in range(num_epochs):
    model.train()
    correct = 0
    total = 0
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs, _ = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_losses.append(running_loss / len(train_loader))
    train_accuracies.append(100 * correct / total)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss / len(train_loader):.4f}, Accuracy: {100 * correct / total:.2f}%")

    # Validation step
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs, _ = model(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    test_losses.append(test_loss / len(test_loader))
    test_accuracies.append(100 * correct / total)
    print(f"Test Loss: {test_loss / len(test_loader):.4f}, Test Accuracy: {100 * correct / total:.2f}%")

# Step viii. Plot loss and accuracy graphs
epochs = range(1, num_epochs + 1)

plt.figure(figsize=(12, 5))

# Loss plot
plt.subplot(1, 2, 1)
plt.plot(epochs, train_losses, label='Training Loss', marker='o')
plt.plot(epochs, test_losses, label='Validation Loss', marker='o')
plt.title('Loss Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid()

# Accuracy plot
plt.subplot(1, 2, 2)
plt.plot(epochs, train_accuracies, label='Training Accuracy', marker='o')
plt.plot(epochs, test_accuracies, label='Validation Accuracy', marker='o')
plt.title('Accuracy Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy (%)')
plt.legend()
plt.grid()

plt.tight_layout()
plt.show()

# Step ix. Visualize feature map
def visualize_feature_map(model, loader):
    model.eval()
    images, _ = next(iter(loader))
    images = images.to(device)
    _, feature_map = model(images)

    feature_map = feature_map[0].cpu().detach()  # First image in batch
    n_filters = feature_map.size(0)

    fig, axes = plt.subplots(1, min(n_filters, 8), figsize=(20, 5))
    for i, ax in enumerate(axes):
        ax.imshow(feature_map[i], cmap='viridis')
        ax.axis('off')
    plt.suptitle('Feature Maps from First Convolutional Layer', fontsize=16)
    plt.show()

visualize_feature_map(model, train_loader)

In [ ]:
import tensorflow_datasets as tfds
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import torchvision

# Step 1: Load dataset using TensorFlow Datasets
(ds_train, ds_test), ds_info = tfds.load(
    'horses_or_humans',
    split=['train', 'test'],
    with_info=True,
    as_supervised=True
)

# Step 2: Wrap TensorFlow dataset for PyTorch compatibility
class TFDSWrapper(torch.utils.data.Dataset):
    def _init_(self, tf_dataset, transform=None):
        self.tf_dataset = list(tfds.as_numpy(tf_dataset))
        self.transform = transform

    def _len_(self):
        return len(self.tf_dataset)

    def _getitem_(self, idx):
        image, label = self.tf_dataset[idx]
        image = Image.fromarray(image)
        if self.transform:
            image = self.transform(image)
        return image, label

# Step 3: Define image transformations
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize images
    transforms.ToTensor(),         # Convert images to PyTorch tensors
    transforms.Normalize((0.5,), (0.5,))  # Normalize image data
])

# Apply transformations to train and test datasets
train_dataset = TFDSWrapper(ds_train, transform=transform)
test_dataset = TFDSWrapper(ds_test, transform=transform)

# Step 4: Create DataLoaders for batching and shuffling
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Step 5: Visualize some images
def imshow(img, labels, class_names):
    img = img / 2 + 0.5  # Undo normalization
    npimg = img.numpy()
    plt.figure(figsize=(10, 5))
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.title("Labels: " + ", ".join([class_names[label] for label in labels]))
    plt.show()

class_names = ['Horse', 'Human']
dataiter = iter(train_loader)
images, labels = next(dataiter)
imshow(torchvision.utils.make_grid(images[:8]), labels[:8], class_names)

# Step 6: Define a simple CNN architecture
class SimpleCNN(nn.Module):
    def _init_(self):
        super(SimpleCNN, self)._init_()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)  # Input: 3 channels (RGB), Output: 16 filters
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1) # Input: 16 filters, Output: 32 filters
        self.fc1 = nn.Linear(32 * 32 * 32, 128)      # Fully connected layer
        self.fc2 = nn.Linear(128, 2)                # Output layer for 2 classes (Horse/Human)
        self.relu = nn.ReLU()                       # Activation function
        self.pool = nn.MaxPool2d(2, 2)              # Pooling layer to downsample

    def forward(self, x):
        x = self.relu(self.conv1(x))  # Convolution 1
        feature_map = x               # Save feature map for visualization
        x = self.pool(feature_map)    # Pooling 1
        x = self.relu(self.conv2(x))  # Convolution 2
        x = self.pool(x)              # Pooling 2
        x = x.view(-1, 32 * 32 * 32)  # Flatten the tensor
        x = self.relu(self.fc1(x))    # Fully connected layer 1
        x = self.fc2(x)               # Fully connected layer 2 (output)
        return x, feature_map

# Step 7: Define loss function and optimizer
model = SimpleCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Step 8: Train the model with accuracy and loss tracking
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

num_epochs = 5
train_losses = []
train_accuracies = []
test_losses = []
test_accuracies = []

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    # Training loop
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()       # Reset gradients
        outputs, _ = model(images)  # Forward pass
        loss = criterion(outputs, labels)  # Compute loss
        loss.backward()             # Backpropagation
        optimizer.step()            # Update weights

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)  # Predictions
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_loss = running_loss / len(train_loader)
    train_accuracy = 100 * correct / total
    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {train_loss:.4f}, Accuracy: {train_accuracy:.2f}%")

    # Validation loop
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs, _ = model(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    test_loss /= len(test_loader)
    test_accuracy = 100 * correct / total
    test_losses.append(test_loss)
    test_accuracies.append(test_accuracy)
    print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%")

# Step 9: Plot training and validation loss and accuracy
epochs = range(1, num_epochs + 1)

plt.figure(figsize=(12, 5))

# Loss plot
plt.subplot(1, 2, 1)
plt.plot(epochs, train_losses, label='Training Loss', marker='o')
plt.plot(epochs, test_losses, label='Validation Loss', marker='o')
plt.title('Loss Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid()

# Accuracy plot
plt.subplot(1, 2, 2)
plt.plot(epochs, train_accuracies, label='Training Accuracy', marker='o')
plt.plot(epochs, test_accuracies, label='Validation Accuracy', marker='o')
plt.title('Accuracy Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy (%)')
plt.legend()
plt.grid()

plt.tight_layout()
plt.show()

# Step 10: Visualize feature maps
def visualize_feature_map(model, loader):
    model.eval()
    images, _ = next(iter(loader))  # Get a batch of images
    images = images.to(device)

    _, feature_map = model(images)  # Get feature maps
    feature_map = feature_map[0].cpu().detach()  # Extract feature map for the first image
    n_filters = feature_map.size(0)

    fig, axes = plt.subplots(1, min(n_filters, 8), figsize=(20, 5))
    for i, ax in enumerate(axes):
        ax.imshow(feature_map[i], cmap='viridis')
        ax.axis('off')
    plt.suptitle('Feature Maps from First Convolutional Layer', fontsize=16)
    plt.show()

# Visualize feature maps
visualize_feature_map(model, train_loader)